In [1]:
import bandit_boxes_env
import datasets
import os
from transformers import AutoTokenizer
import numpy as np

import importlib
importlib.reload(bandit_boxes_env)

/n/home12/cfpark00/anaconda3/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.5.1 available.


<module 'bandit_boxes_env' from '/n/home12/cfpark00/ML/llm-meta-rl/environments/bandit_boxes_env.py'>

In [2]:
local_dir = '../data/datasets/multiboxes'
os.makedirs(local_dir, exist_ok=True)

In [3]:
split='bandit_boxes_n_rows=128_n_boxes=2_max_games=16'
dataset=bandit_boxes_env.make_dataset(n_rows=128,n_boxes=2,max_games=16)
ds=datasets.Dataset.from_list(dataset)
ds.to_parquet(os.path.join(local_dir,f"{split}.parquet"))
ds.to_json(os.path.join(local_dir,f"{split}.json"))
ds

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 181.73ba/s]


Dataset({
    features: ['prompt', 'hidden_params'],
    num_rows: 128
})

In [8]:
pf=os.path.join(local_dir,f"{split}.parquet")
df=datasets.Dataset.from_parquet(pf)
df[0]["hidden_params"]["p_binoms"]

[0.1903490245846383, 0.8096509754153617]

In [17]:
import pandas as pd
df=pd.read_parquet(pf,dtype=object)
df["hidden_params"][0]["p_binoms"]

TypeError: read_table() got an unexpected keyword argument 'dtype'

In [12]:
messages=[
    {"role":"assistant","content":"I want to play a game."},
]
messages=[
    {"role":"assistant","content":"Reasoning: None\nFinal Choice: 1"},
]
hidden_params={
    "p_binoms": [0.9, 0.01],
    "max_games": 16,
}
bandit_boxes_env.get_response_message(messages,hidden_params)

{'role': 'user',
 'content': 'Reward: 1\n\nRound 2: Box 1 vs. Box 2?',
 'done': False}

In [10]:
messages=[
    {"role":"assistant","content":"Reasoning: None\nFinal Choice: 2"},
]
tokenizer=AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

In [13]:
import requests
messages=[
    {"role":"assistant","content":"Reasoning: None\nFinal Choice: 1"},
]
text=tokenizer.apply_chat_template(messages,tokenize=False)
payload = {
    "text": [text],
    "hidden_params": [hidden_params],
}
print("ENV RESPONSE:",requests.post('http://holy8a29407:5001/get_env_response', json=payload).json())

messages=[
    {"role":"user","content":"Reward: 0."},
    {"role":"user","content":"Reward: 1"},
]
text=tokenizer.apply_chat_template(messages,tokenize=False)
payload = {
    "text": [text],
    "hidden_params": [hidden_params],
}
print("REWARD:",requests.post('http://holy8a29407:5001/get_reward', json=payload).json())

ENV RESPONSE: [{'env_response_message': {'role': 'user', 'content': 'Reward: 1\n\nRound 2: Box 1 vs. Box 2?', 'done': False}}]
REWARD: {'reward': [1.0]}
